In [171]:
import re
import string

import pandas as pd
import numpy as np

import transformers
import torch

from nltk import WordNetLemmatizer

import faiss

from tqdm.notebook import tqdm

from towhee import pipeline

from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import  paired_distances

# Загрузка и подготовка данных

In [128]:
data = pd.read_csv('../data/products.csv')
data = data.drop_duplicates()
tokenizer = transformers.BertTokenizer('../model/vocab.txt')
try:
	embedded_description = pd.read_csv('../data/embedded_description')
	embedded_product_composition = pd.read_csv('../data/embedded_product_composition')
	embedded_product_usage = pd.read_csv('../data/embedded_product_usage')
	embedded_3_in_1 = pd.read_csv('../data/embedded_3_in_1')
except:
	pass

# model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel,
													# transformers.DistilBertTokenizer,
													# 'distilbert-base-uncased')

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11910 entries, 0 to 11909
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   11910 non-null  int64  
 1   sku                  11910 non-null  object 
 2   name                 11910 non-null  object 
 3   brand                11906 non-null  object 
 4   brand_type           11910 non-null  object 
 5   dimension17          11513 non-null  object 
 6   dimension18          11474 non-null  object 
 7   dimension19          1385 non-null   object 
 8   dimension20          1770 non-null   object 
 9   country              10818 non-null  object 
 10  price                11910 non-null  int64  
 11  old_price            11910 non-null  int64  
 12  category_type        11908 non-null  object 
 13  url                  11910 non-null  object 
 14  images               11910 non-null  object 
 15  type                 11910 non-null 

Один товар может принадлежать нескольким категориям. Если бы у нас была информация о популярности товаров в той или иной категории, то можно было бы удалить товары из категорий, где они не пользуются спросом. Но т.к. такой информации у нас нет, просто удалим такие товары-дубликаты.

In [3]:
index_to_del = data[data.drop(['category', 'category_ru', 'product_usage', 'product_composition', 'description'], axis=1).duplicated()].index
display(f'Данных до удаления: {data.shape}')
data = data.drop(index_to_del)
display(f'Данных после удаления: {data.shape}')

'Данных до удаления: (11910, 27)'

'Данных после удаления: (11910, 27)'

In [4]:
def text_processing(text: str) -> str:
	# оставляем пропуски без изменений
	if text is np.nan:
		return np.nan
	# приводим текст к нижнему регистру
	text = text.lower()
	# заменяем символы и знаки пунктуации
	text = re.sub('\(.*?\)', '', text)
	trans_dict = str.maketrans('', '', string.punctuation)
	text = text.translate(trans_dict)
	# избавляемся от лишних пробелов
	text = ' '.join(text.split())

	return text

In [5]:
text_columns = ['description', 'product_usage', 'product_composition']

for column in text_columns:
	data[column] = data[column].apply(text_processing)

data.head()

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_sku,main_product_id,best_loyalty_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
0,151709,19760331641,анклет,Nothing but Love,standard,Браслет,Женский,NaN,NaN,NaN,...,19760331641,151709,476.0,False,False,NaN,NaN,NaN,ukrashenija,украшения
1,219259,19000070237,Peptide 9 Balance Eye Hyaluronic Volumy Eye Cr...,MEDI PEEL,standard,Крем,Унисекс,Против признаков старения,Глаза,Южная Корея,...,19000070237,219259,2763.0,False,False,NaN,NaN,NaN,uhod,уход
2,227749,19000042157,Mad Maxcara,RAD,standard,Объемная тушь для ресниц,NaN,NaN,NaN,Нидерланды,...,19000042157,227749,NaN,False,True,это она немного крэйзи подруга твоих ресниц ту...,aqua synthetic beeswax paraffin ci 77499 acaci...,rad — креативный beauty бренд для самых прогре...,makijazh,макияж
3,151711,19760331652,Утонченная Мари,Nothing but Love,standard,Браслет,Женский,NaN,NaN,NaN,...,19760331652,151711,644.0,False,False,NaN,NaN,NaN,ukrashenija,украшения
4,155448,19760318176,TIME CONTROL +,TALIKA,standard,Массажер,Женский,NaN,NaN,Франция,...,19760318176,155448,13837.0,False,True,косметический прибор,включите прибор нажав кнопку и подождите пока ...,абстерполимер термопластический эластомер usbк...,tehnika,техника


In [6]:
def lower(text: str) -> str:
	if text is np.nan:
		return np.nan
	return text.lower()

In [7]:
columns_to_lower = ['name', 'brand', 'dimension17', 'dimension18', 'dimension19', 'dimension20', 'country', 'category_type']

for column in columns_to_lower:
	data[column] = data[column].apply(lower)

data.head()

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_sku,main_product_id,best_loyalty_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
0,151709,19760331641,анклет,nothing but love,standard,браслет,женский,NaN,NaN,NaN,...,19760331641,151709,476.0,False,False,NaN,NaN,NaN,ukrashenija,украшения
1,219259,19000070237,peptide 9 balance eye hyaluronic volumy eye cr...,medi peel,standard,крем,унисекс,против признаков старения,глаза,южная корея,...,19000070237,219259,2763.0,False,False,NaN,NaN,NaN,uhod,уход
2,227749,19000042157,mad maxcara,rad,standard,объемная тушь для ресниц,NaN,NaN,NaN,нидерланды,...,19000042157,227749,NaN,False,True,это она немного крэйзи подруга твоих ресниц ту...,aqua synthetic beeswax paraffin ci 77499 acaci...,rad — креативный beauty бренд для самых прогре...,makijazh,макияж
3,151711,19760331652,утонченная мари,nothing but love,standard,браслет,женский,NaN,NaN,NaN,...,19760331652,151711,644.0,False,False,NaN,NaN,NaN,ukrashenija,украшения
4,155448,19760318176,time control +,talika,standard,массажер,женский,NaN,NaN,франция,...,19760318176,155448,13837.0,False,True,косметический прибор,включите прибор нажав кнопку и подождите пока ...,абстерполимер термопластический эластомер usbк...,tehnika,техника


Уберем лишние знаки из ссылок с изображениями

In [8]:
def url_processing(text: str) -> str:
	return text.replace('"', '').replace('\'', '').replace('[', '').replace(']', '')

data['images'] = data['images'].apply(url_processing)

# Генерация эмбеддингов

In [ ]:
def lemmatization(data):
	if data is np.nan:
		return ''
	return ' '.join([WordNetLemmatizer().lemmatize(word) for word in data.split()])

Загрузим RuBERT для генерации эмбеддингов

In [9]:
config = transformers.BertConfig.from_json_file(
	'../model/bert_config.json')
model = transformers.BertModel.from_pretrained(
	'../model/pytorch_model.bin', config=config).to('cuda:0')

Some weights of the model checkpoint at ../model/pytorch_model.bin were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
batch_size = 1

for column in text_columns:
	text = data[column].fillna('')
	# lemmas = data[column].apply(lemmatization)
	vector = text.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512))
	# применим padding к векторам
	n = len(max(vector, key=len))
	# англ. вектор с отступами
	padded = np.array([i + [0]*(n - len(i)) for i in vector.values])

	# создадим маску для важных токенов
	attention_mask = np.where(padded != 0, 1, 0)

	embeddings = []
	for i in tqdm(range(padded.shape[0] // batch_size)):
		# преобразуем данные
		batch = torch.LongTensor(padded[batch_size*i : batch_size*(i+1)]).to('cuda:0')
		# преобразуем маску
		attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i : batch_size*(i+1)]).to('cuda:0')
		with torch.no_grad():
			batch_embeddings = model(batch, attention_mask=attention_mask_batch)

		# преобразуем элементы методом numpy() к типу numpy.array
		embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

	features = pd.DataFrame(np.concatenate(embeddings))
	features.to_csv(f'../data/embedded_{column}', index=False)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


  0%|          | 0/11910 [00:00<?, ?it/s]

  0%|          | 0/11910 [00:00<?, ?it/s]

  0%|          | 0/11910 [00:00<?, ?it/s]

In [11]:
batch_size = 1
text = []
# заменим пропуски в полях на пустую строку, для корректной генерации токенов
for d, pu, pc in data[['description', 'product_usage', 'product_composition']].values:
	if d is np.nan:
		d = ' '
	if pu is np.nan:
		pu = ' '
	if pc is np.nan:
		pc = ' '
	text.append(d + pu + pc)

data['3_in_1'] = text
text = data['3_in_1'].fillna('')
# lemmas = data['3_in_1'].apply(lemmatization)
vector = text.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512))
# применим padding к векторам
n = len(max(vector, key=len))
# англ. вектор с отступами
padded = np.array([i + [0]*(n - len(i)) for i in vector.values])

# создадим маску для важных токенов
attention_mask = np.where(padded != 0, 1, 0)

embeddings = []
for i in tqdm(range(padded.shape[0] // batch_size)):
	# преобразуем данные
	batch = torch.LongTensor(padded[batch_size*i : batch_size*(i+1)]).to('cuda:0')
	# преобразуем маску
	attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i : batch_size*(i+1)]).to('cuda:0')
	with torch.no_grad():
		batch_embeddings = model(batch, attention_mask=attention_mask_batch)

	# преобразуем элементы методом numpy() к типу numpy.array
	embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

features = pd.DataFrame(np.concatenate(embeddings))

features.to_csv(f'../data/embedded_3_in_1', index=False)

  0%|          | 0/11910 [00:00<?, ?it/s]

### Сформируем индексы faiss на основе эмбеддингов описания товаров

1. Создадим индекс для description

In [13]:
index = faiss.IndexFlatL2(embedded_description.shape[1])
print(index.ntotal)  # пока индекс пустой
index.add(np.ascontiguousarray(embedded_description.to_numpy().astype('float32')))
print(index.ntotal)  # теперь в нем n векторов

0
11910


Сохраним индексы в файл

In [25]:
faiss.write_index(index, 'faiss_description_index.index')

Проверим как работает поиск по данному индексу

Получим предсказания по индексам

In [23]:
topn = 10
product_index_in_data = 1474
distances, same_embedding_indexes = index.search(np.ascontiguousarray(embedded_description.to_numpy().astype('float32')[product_index_in_data].reshape((1, -1))), 10)
print(same_embedding_indexes[0]) # индексы самых похожих векторов
print(distances) # расстояния, отсортированные по убыванию

[1474 3421 2136 1419  259 6292 4143 4404 3040  305]
[[ 0.       32.145218 35.11202  37.72622  40.14955  40.799274 42.39068
  42.751522 43.09414  43.523987]]


In [26]:
data.iloc[same_embedding_indexes[0]]

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_sku,main_product_id,best_loyalty_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
1474,84662,89190400038,Refreshing Eye Contour Cream,EAU THERMALE AVENE,standard,Крем,Женский,Против признаков старения,Глаза,Франция,...,89190400038,84662,1483.0,False,False,Для гиперчувствительной области.,Наносить днем легкими массирующими движениями.,AVENE THERMAL SPRING WATER (AVENE AQUA). MINER...,zdorov-e-i-apteka,здоровье и аптека
3421,107554,15240700012,Sisleya L’Integral Anti-Age Anti-Wrinkle Conce...,Sisley,standard,Сыворотки,Женский,Против признаков старения,Лицо,Франция,...,15240700012,107554,39520.0,False,False,Антивозрастная интегральная сыворотка для лица,Небольшое количество сыворотки (1-2 нажатия на...,Французский бренд Sisley основан в 1976 году Ю...,uhod,уход
2136,32055,99720100031,WRINKLE COLLAGEN FEELTOX AMPOULE,The Skin House,standard,Сыворотки,Унисекс,Против признаков старения,Лицо,Южная Корея,...,99720100031,32055,2084.0,False,False,Сыворотка ампульная с коллагеном,"Вода, гиалуронат натрия, глицерин, 1,3 бутилен...",NaN,ini-formaty,тревел-форматы
1419,208712,19000060885,Peptide Anti-aging Series,EVERTY,standard,Сыворотки,Унисекс,Против признаков старения,Лицо,Россия,...,19000060885,208712,NaN,True,False,Пептидная сыворотка для всех типов кожи,Небольшое количество средства аккуратно нанест...,"Трипептид Syn-Coll, Olivem 2020, Sensolen, гер...",uhod,уход
259,135921,83510100014,After Shaving Balm Sensitive Skin,Proraso,standard,Бальзам,Мужской,Для бритья,Лицо,Италия,...,83510100014,135921,1501.0,False,False,Бальзам после бритья для чувствительной кожи,Нанести на область бритья плавными круговыми д...,NaN,dlja-muzhchin,для мужчин
6292,118036,10030-14146900001,Phyto-Cernes Eclat,Sisley,standard,Консилеры,Женский,NaN,NaN,Франция,...,14146900001,34321,NaN,True,False,Корректор для кожи вокруг глаз,"Перед нанесением фитокорректора ""Сияние"" на ко...",Экстракт красного винограда: тонизирует; Экстр...,makijazh,макияж
4143,119171,99970200020,Starting Treatment Eye Cream,Secret Key,standard,Крем,Унисекс,Против признаков старения,Глаза,Южная Корея,...,99970200020,119171,1096.0,False,False,Ферментированный крем для глаз,нанесите немного восстанавливающего крема на о...,NaN,ini-formaty,тревел-форматы
4404,193165,19000026991,FLEUR d'ORANGER,SOLINOTES,standard,Парфюмерная вода,Унисекс,NaN,NaN,Франция,...,19000026991,193165,NaN,True,True,В чувственном цветочно-цитрусовом аромате,только для наружного применения.,"SOLINOTES – французский парфюмерный бренд, кот...",parfjumerija,парфюмерия
3040,34755,15240400012,Sisleya L’Integral Anti-Age extra-riche,Sisley,standard,Крем,Унисекс,Против признаков старения,Лицо,Франция,...,15240400012,34755,41600.0,False,False,Антивозрастной крем для сухой кожи,Наносить утром и вечером на чистую кожу лица и...,Экстракт косточек яблока (фитостимулины и урсу...,uhod,уход
305,143613,89630900006,Ceramides Mask,LABORATORIUM,standard,Маски кремовые,Женский,Увлажнение и питание,Лицо,Россия,...,89630900006,143613,307.0,False,False,В составе этой маски - питательное аргановое м...,Тонким слоем равномерно нанесите маску на пред...,"вода, аргановое масло, глицерин, пропиленглико...",organika,органика


2. Создадим индекс по составу продукта

In [27]:
index = faiss.IndexFlatL2(embedded_product_composition.shape[1])
print(index.ntotal)  # пока индекс пустой
index.add(np.ascontiguousarray(embedded_product_composition.to_numpy().astype('float32')))
print(index.ntotal)  # теперь в нем n векторов

0
11910


Сохраним индекс в файл

In [28]:
faiss.write_index(index, 'faiss_product_composition_index.index')

Получим 10 ближайших для какого-либо продукта

In [29]:
topn = 10
product_index_in_data = 1474
distances, same_embedding_indexes = index.search(np.ascontiguousarray(embedded_product_composition.to_numpy().astype('float32')[product_index_in_data].reshape((1, -1))), 10)
print(same_embedding_indexes[0]) # индексы самых похожих векторов
print(distances) # расстояния, отсортированные по убыванию

[1474  580 1649  644 2200 1583 1557 3096 5896  768]
[[ 0.       12.832222 13.263858 13.62476  13.65633  13.773034 13.786741
  13.792073 14.219757 14.237248]]


Посмотрим да рекомендацию

In [30]:
data.iloc[same_embedding_indexes[0]]

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_sku,main_product_id,best_loyalty_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
1474,84662,89190400038,Refreshing Eye Contour Cream,EAU THERMALE AVENE,standard,Крем,Женский,Против признаков старения,Глаза,Франция,...,89190400038,84662,1483.0,False,False,Для гиперчувствительной области.,Наносить днем легкими массирующими движениями.,AVENE THERMAL SPRING WATER (AVENE AQUA). MINER...,zdorov-e-i-apteka,здоровье и аптека
580,225541,19000053765,Urban Eco Harakeke Root Toner,THE Saem,standard,Тонер,Женский,Увлажнение и питание,Лицо,Южная Корея,...,19000053765,225541,1631.0,False,False,"Тонер нежно обволакивает грубую кожу, интенсив...",нанесите небольшое количество тонера на ватный...,"Phormium Tenax Root Extract, Propanediol, 1,2-...",azija,азия
1649,171588,19000003067,BABY Basic Line With Calendula & Panthenol,roofa spain,standard,Лосьон,Для детей,NaN,Тело,NaN,...,19000003067,171588,719.0,False,True,Натуральный лосьон для тела,используйте после купания или очищения кожи. Н...,"Aqua (Thermal Water), Caprylic/ Capric Triglyc...",dlja-detej,для детей
644,163627,10019-19000004370,Poudre De Beauté Mat Naturel,GUCCI,middle,Компактная пудра,Женский,NaN,NaN,Италия,...,19000004670,163728,5112.0,False,False,Матирующая компактная пудра Poudre De Beauté M...,"Благодаря спонжу, состоящему из 2 видов микроф...","Talc, Zinc Stearate, Zea Mays (Corn) Starch, M...",makijazh,макияж
2200,161326,19760320732,Elasticity & lifting Moisture Mask Pack,Cosworker,standard,Маски тканевые,Унисекс,Против признаков старения,Лицо,Южная Корея,...,19760320732,161326,NaN,True,False,Маска,Нанесите маску на кожу лица и оставьте на 15-2...,"D.I-water, Glycerin, Aloe Barbadensis Leaf Ext...",uhod,уход
1583,161283,19760320731,Moisturizing Mask Pack,Cosworker,standard,Маски тканевые,Унисекс,Увлажнение и питание,Лицо,Южная Корея,...,19760320731,161283,NaN,True,False,Маска,Нанесите маску на кожу лица и оставьте на 15-2...,"D.I-water, Glycerin, Butylene Glycol, Aloe Bar...",uhod,уход
1557,166713,19760343848,NATURAL MOISTURE MASK SHEET - MADECASSOSIDE,ORJENA,standard,Маски тканевые,Унисекс,Увлажнение и питание,Лицо,Южная Корея,...,19760343848,166713,NaN,True,True,Маска,1. После очищения лица нанесите тонер для подг...,"Water, Glycerin, Propylene Glycol, Butylene Gl...",uhod,уход
3096,66919,89190400028,Revitalizing Nourishing Cream,EAU THERMALE AVENE,standard,Крем,Унисекс,Увлажнение и питание,Лицо,Франция,...,89190400028,66919,2370.0,False,False,Помогает восстановить естественную красоту сух...,Наносить на очищенную кожу утром и/или вечером.,"AVENE AQUA, GLYCERIN, OCTYLDODECANOL, PARAFFIN...",zdorov-e-i-apteka,здоровье и аптека
5896,153249,19760332149,Milk,Bubchen,standard,Молочко,Для детей,NaN,Тело,Германия,...,19760332149,153249,NaN,True,False,"Молочко увлажняет и смягчает кожу, сохраняет э...","Для ухода за кожей, можно использовать для мас...","Aqua, Helianthus Annuus Seed Oil, Sorbitol, Oc...",dlja-detej,для детей
768,203450,10693-15370600002,Creamy Eye Treatment with Avocado,Kiehl's,standard,Крем,Женский,Увлажнение и питание,Глаза,США,...,15370600002,213818,NaN,False,False,№1 Крем для кожи вокруг глаз *,Используйте крем утром и/или вечером.,AQUA / WATER BUTYROSPERMUM PARKII BUTTER / SHE...,uhod,уход


3. Построим индекс по инструкциям продукта

In [32]:
index = faiss.IndexFlatL2(embedded_product_usage.shape[1])
print(index.ntotal)  # пока индекс пустой
index.add(np.ascontiguousarray(embedded_product_usage.to_numpy().astype('float32')))
print(index.ntotal)  # теперь в нем n векторов

0
11910


In [33]:
faiss.write_index(index, 'faiss_product_usage_index.index')

In [36]:
topn = 10
product_index_in_data = 4732
distances, same_embedding_indexes = index.search(np.ascontiguousarray(embedded_product_usage.to_numpy().astype('float32')[product_index_in_data].reshape((1, -1))), 10)
print(same_embedding_indexes[0]) # индексы самых похожих векторов
print(distances) # расстояния, отсортированные по убыванию

[2925 4732 2963 4715 3910 4712 4639 4840 4660 4507]
[[0.       0.       0.       2.066492 2.066492 2.066492 2.066492 2.066492
  2.066492 2.066492]]


In [37]:
data.iloc[same_embedding_indexes[0]]

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_sku,main_product_id,best_loyalty_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
2925,188967,12241-19000033554,черный,Belle YOU,standard,Легинсы,Женский,NaN,NaN,Россия,...,19000033554,188966,NaN,False,False,Фактурные легинсы из мягкого трикотажа в рубчи...,"96% полиамид, 4% эластан","Бренд Belle YOU создает белье и одежду, в кото...",odezhda-i-aksessuary,одежда и аксессуары
4732,222558,10689-19000071649,ENERGETIC,Mirey,standard,Колготки,Женский,NaN,NaN,Россия,...,19000071649,222557,297.0,False,True,Фантазийные колготки с геометрическим узором.,"96% полиамид, 4% эластан","Мы наконец-то живем в эпоху, когда удивлять и ...",nizhnee-bel-jo,нижнее бельё
2963,188963,12241-19000033551,темно-коричневый,Belle YOU,standard,Легинсы,Женский,NaN,NaN,Россия,...,19000033551,188962,NaN,False,False,Фактурные легинсы из мягкого трикотажа в рубчи...,"96% полиамид, 4% эластан","Бренд Belle YOU создает белье и одежду, в кото...",nizhnee-bel-jo,нижнее бельё
4715,222707,19000071484,Spalla Stretta mahgo,MY,standard,Топ,Женский,NaN,NaN,Россия,...,19000071484,222707,711.0,False,True,Женский базовый топ на тонких бретелях из мягк...,"92% полиамид, 8% эластан","Белье, которое реально удобно носить.",nizhnee-bel-jo,нижнее бельё
3910,222577,10689-19000071663,SYMPATHY,Mirey,standard,Чулки,Женский,NaN,NaN,Россия,...,19000071663,222576,547.0,False,True,Шелковистые полуматовые чулки с резинкой на си...,"92% полиамид, 8% эластан","Мы наконец-то живем в эпоху, когда удивлять и ...",nizhnee-bel-jo,нижнее бельё
4712,222702,19000071479,Spalla Stretta moon,MY,standard,Топ,Женский,NaN,NaN,Россия,...,19000071479,222702,711.0,False,True,Женский базовый топ на тонких бретелях из мягк...,"92% полиамид, 8% эластан","Белье, которое реально удобно носить.",nizhnee-bel-jo,нижнее бельё
4639,222704,19000071481,Spalla Stretta bodily,MY,standard,Топ,Женский,NaN,NaN,Россия,...,19000071481,222704,711.0,False,True,Женский базовый топ на тонких бретелях из мягк...,"92% полиамид, 8% эластан","Белье, которое реально удобно носить.",nizhnee-bel-jo,нижнее бельё
4840,222699,19000071476,Spalla Stretta bianco,MY,standard,Топ,Женский,NaN,NaN,Россия,...,19000071476,222699,711.0,False,True,Женский базовый топ на тонких бретелях из мягк...,"92% полиамид, 8% эластан","Белье, которое реально удобно носить.",nizhnee-bel-jo,нижнее бельё
4660,222701,19000071478,Spalla Stretta iris,MY,standard,Топ,Женский,NaN,NaN,Россия,...,19000071478,222701,711.0,False,True,Женский базовый топ на тонких бретелях из мягк...,"92% полиамид, 8% эластан","Белье, которое реально удобно носить.",nizhnee-bel-jo,нижнее бельё
4507,222710,19000071487,Spalla Stretta ocean,MY,standard,Топ,Женский,NaN,NaN,Россия,...,19000071487,222710,711.0,False,True,Женский базовый топ на тонких бретелях из мягк...,"92% полиамид, 8% эластан","Белье, которое реально удобно носить.",nizhnee-bel-jo,нижнее бельё


### Сгенерируем эмбеддинги изображений

In [197]:
image_embeddings = pd.DataFrame()
images_data = pd.read_csv('../data/product_images.csv')
embedding_pipeline = pipeline('towhee/image-embedding-resnet50', )

In [199]:
pbar = tqdm(range(len(images_data)))
for index, row in images_data.head(20).iterrows():
	img_path = rf'P:\market_recommendations\data\images\{row["image"]}'
	embedding = pd.DataFrame(embedding_pipeline(img_path).reshape((1, -1)))
	embedding['id'] = row['id']
	embedding['sku'] = row['sku']
	image_embeddings = pd.concat([image_embeddings, embedding])
	pbar.update(1)

  0%|          | 0/26278 [00:00<?, ?it/s]

In [206]:
data.head()

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_sku,main_product_id,best_loyalty_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
0,151709,19760331641,анклет,Nothing but Love,standard,Браслет,Женский,NaN,NaN,NaN,...,19760331641,151709,476.0,False,False,NaN,NaN,NaN,ukrashenija,украшения
1,219259,19000070237,Peptide 9 Balance Eye Hyaluronic Volumy Eye Cr...,MEDI PEEL,standard,Крем,Унисекс,Против признаков старения,Глаза,Южная Корея,...,19000070237,219259,2763.0,False,False,NaN,NaN,NaN,uhod,уход
2,227749,19000042157,Mad Maxcara,RAD,standard,Объемная тушь для ресниц,NaN,NaN,NaN,Нидерланды,...,19000042157,227749,NaN,False,True,"Это ОНА, немного крэйзи подруга твоих ресниц, ...","Aqua (Water), Synthetic Beeswax, Paraffin, CI ...",RAD — креативный beauty бренд для самых прогре...,makijazh,макияж
3,151711,19760331652,Утонченная Мари,Nothing but Love,standard,Браслет,Женский,NaN,NaN,NaN,...,19760331652,151711,644.0,False,False,NaN,NaN,NaN,ukrashenija,украшения
4,155448,19760318176,TIME CONTROL +,TALIKA,standard,Массажер,Женский,NaN,NaN,Франция,...,19760318176,155448,13837.0,False,True,Косметический прибор,"включите прибор, нажав кнопку, и подождите, по...",АБС-терполимер Термопластический эластомер USB...,tehnika,техника


In [200]:
embedding

,0,1,2,3,4,5,6,7,8,9,...,2040,2041,2042,2043,2044,2045,2046,2047,id,sku
0,0.0,0.0,0.001977,0.0,0.011048,0.0,0.114332,0.073874,0.0,0.0,...,0.0,0.012779,0.005214,0.0,0.092239,0.016707,0.246688,0.0,161955,19000007001


In [96]:
image_embeddings.to_csv('images_embeddings.csv', index=False)

In [97]:
image_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,2040,2041,2042,2043,2044,2045,2046,2047,id,sku
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,151709,19760331641
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,151709,19760331641
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,219259,19000070237
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,219259,19000070237
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,219259,19000070237


Создадим индексы для эмбеддингов картинок

In [184]:
image_embeddings_to_faiss = image_embeddings.drop(['id', 'sku'], axis=1)
# pca = PCA(n_components=50, random_state=25)
# image_embeddings_to_faiss = pd.DataFrame(pca.fit_transform(image_embeddings_to_faiss))
index = faiss.IndexFlatL2(image_embeddings_to_faiss.shape[1])
print(index.ntotal)  # пока индекс пустой
index.add(np.ascontiguousarray(image_embeddings_to_faiss.to_numpy().astype('float32')))
print(index.ntotal)  # теперь в нем n векторов

0
26308


In [137]:
faiss.write_index(index, 'faiss_image.index')

In [170]:
topn = 10
product_index_in_data = 1
distances, same_embedding_indexes = index.search(np.ascontiguousarray(image_embeddings_to_faiss.to_numpy().astype('float32')[product_index_in_data].reshape((1, -1))), 10)
print(same_embedding_indexes[0]) # индексы самых похожих векторов
print(distances) # расстояния, отсортированные по убыванию

[    1 26307 26306 26305 26304    38    56    58    63    61]
[[0.0000000e+00 5.2419482e-23 5.2427544e-23 5.2427544e-23 5.2427544e-23
  5.4285671e-23 6.0406544e-23 6.0406967e-23 6.0406980e-23 6.0406980e-23]]


In [ ]:
for sku in image_embeddings.iloc[same_embedding_indexes[0]]['sku']:
	display(data[data['sku'] == sku])

In [129]:
data.head()

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_sku,main_product_id,best_loyalty_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
0,151709,19760331641,анклет,Nothing but Love,standard,Браслет,Женский,NaN,NaN,NaN,...,19760331641,151709,476.0,False,False,NaN,NaN,NaN,ukrashenija,украшения
1,219259,19000070237,Peptide 9 Balance Eye Hyaluronic Volumy Eye Cr...,MEDI PEEL,standard,Крем,Унисекс,Против признаков старения,Глаза,Южная Корея,...,19000070237,219259,2763.0,False,False,NaN,NaN,NaN,uhod,уход
2,227749,19000042157,Mad Maxcara,RAD,standard,Объемная тушь для ресниц,NaN,NaN,NaN,Нидерланды,...,19000042157,227749,NaN,False,True,"Это ОНА, немного крэйзи подруга твоих ресниц, ...","Aqua (Water), Synthetic Beeswax, Paraffin, CI ...",RAD — креативный beauty бренд для самых прогре...,makijazh,макияж
3,151711,19760331652,Утонченная Мари,Nothing but Love,standard,Браслет,Женский,NaN,NaN,NaN,...,19760331652,151711,644.0,False,False,NaN,NaN,NaN,ukrashenija,украшения
4,155448,19760318176,TIME CONTROL +,TALIKA,standard,Массажер,Женский,NaN,NaN,Франция,...,19760318176,155448,13837.0,False,True,Косметический прибор,"включите прибор, нажав кнопку, и подождите, по...",АБС-терполимер Термопластический эластомер USB...,tehnika,техника


In [189]:
x = image_embeddings_to_faiss[0].to_numpy().reshape((1, -1))
paired_distances(x, image_embeddings_to_faiss.T, metric='cosine')

ValueError: X and Y should be of same shape. They were respectively (1, 26308) and (2048, 26308) long.

In [194]:
image_embeddings[:50]

,0,1,2,3,4,5,6,7,8,9,...,2040,2041,2042,2043,2044,2045,2046,2047,id,sku
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,151709,19760331641
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,151709,19760331641
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,219259,19000070237
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,219259,19000070237
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,219259,19000070237
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,227749,19000042157
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,227749,19000042157
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,227749,19000042157
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,151711,19760331652
0,0.025007,0.0,0.000875,0.0,0.171166,0.0,0.004925,0.254992,0.067476,0.0,...,0.004248,0.097569,0.0,0.183973,0.011834,0.029646,0.341438,0.0,151711,19760331652
